In [22]:
import tensorflow as tf
import math
import numpy as np
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import itertools
from nltk.tokenize import RegexpTokenizer
from functools import partial

In [20]:
# bible
def sparse(x):
    t = [0] * vocab_size
    t[x] = 1
    return t
def process(x):
    t = [sparse(w_i[i]) for i in x.split()]
    return t

pairs = np.load('pairs.npy')
w_i = np.load('word2index.npy').item()
i_w = np.load('index2word.npy')
vocab_size = len(i_w)
pairs.shape

(110506, 2)

In [23]:
# conv.txt
tokenizer = RegexpTokenizer(r'\w+')
def process_sentence(x, eos=False):
#     x = x.split()
    x = tokenizer.tokenize(x)
    x = [i.lower() for i in x]
    if eos:
        x = ['eos'] + x + ['eos']
    return x

def one_hot(x):
    t = [0] * vocab_size
    t[x] = 1
    return t
    
def sent_encode(x):
    x = [w_i[i] for i in x]
    x = map(one_hot, x)
    return x

data=pd.read_csv('conv.txt', delimiter='\t', error_bad_lines=False, header=None)
data.dropna(inplace=True)
inp = map(partial(process_sentence), data[0].values)
out = map(partial(process_sentence, eos=True), data[1].values)
# inp=map(process_sentence, data[0].values)
# out=map(process_sentence, data[1].values)
words=np.r_[inp, out]
# words=np.array([w.split() for w in words])
words = list(set(itertools.chain(*words)))
vocab_size=len(words)
i_w = dict(enumerate(words))
w_i = {v:k for k, v in i_w.items()}
inp = map(sent_encode, inp)
out = map(sent_encode, out)

Skipping line 1681: expected 2 fields, saw 3
Skipping line 1765: expected 2 fields, saw 3
Skipping line 1984: expected 2 fields, saw 3
Skipping line 2046: expected 2 fields, saw 3
Skipping line 2086: expected 2 fields, saw 3
Skipping line 2313: expected 2 fields, saw 3
Skipping line 2314: expected 2 fields, saw 3



In [25]:
input_size = vocab_size
print input_size
dims = [300, 200, 300]
num_layers = 3

3327


In [26]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxz']) + tf.matmul(p, enc_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxr']) + tf.matmul(p, enc_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, enc_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), enc_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

def recurrence2(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxz']) + tf.matmul(p, dec_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxr']) + tf.matmul(p, dec_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, dec_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), dec_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

In [27]:
tf.reset_default_graph()
with tf.device('/cpu:0'):
    with tf.name_scope('encoder'):
        enc_layers = []
        incoming = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='encoder_incoming')
        enc_layers.append({'states': incoming, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [enc_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            layer['initial'] = tf.placeholder(shape=[dims[j-1]], dtype=tf.float32, name='initial'+str(j))
            enc_layers.append(layer)
            layer['states'] = tf.scan(recurrence, enc_layers[-2]['states'], initializer=layer['initial'], name='encoder_states'+str(j))
            layer['last'] = tf.squeeze(tf.expand_dims(tf.nn.embedding_lookup(layer['states'], 
                                                                             tf.shape(layer['states'])[0] - 1), dim=0),
                                      name='last'+str(j))

In [28]:
with tf.device('/cpu:0'):
    with tf.name_scope('decoder'):
        
        dec_layers = []
        decoder_input = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='decoeder_input')
        dec_layers.append({'states': decoder_input, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [dec_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            dec_layers.append(layer)
            
            layer['states'] = tf.scan(recurrence2, dec_layers[-2]['states'], initializer=enc_layers[j]['last'], 
                                      name='decoder_states'+str(j))
            

        # softmax layer
        Why = tf.Variable(tf.random_normal(
            [dims[-1], input_size],
            stddev=1.0 / math.sqrt(dims[-1])), dtype=tf.float32)
        by = tf.Variable(tf.zeros(input_size), dtype=tf.float32)
        outputs = tf.matmul(dec_layers[-1]['states'], Why) + by
        
        # decoding step (with 'teacher forcing')

        decoder_target = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='decoder_target')
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(outputs, decoder_target))
        optimizer_op = tf.train.AdagradOptimizer(0.1).minimize(loss)

In [29]:
saver = tf.train.Saver()
sess =  tf.Session()
# saver.restore(sess, 'checkpoint.ckpt')
sess.run(tf.initialize_all_variables())
ix = 0
avg_loss = 0

In [32]:
def beam_search(N=5):
    ''' greedy search to find top N outputs'''
    pass

def generate_out(inp):
    inp = process_sentence(inp)
    inp = sent_encode(inp)
    out = ['eos']
    seed = [one_hot(w_i['eos'])]
    while len(seed) < 20:
        feed = {k:v for k,v in zip([l['initial'] for l in enc_layers[1:]], [np.zeros(d) for d in dims])}
        feed.update({enc_layers[0]['states'] : inp, dec_layers[0]['states']: seed})
        o = sess.run(outputs, feed_dict=feed)[-1]
        r = i_w[np.argmax(o)]
        seed.append(o)
        out.append(r)
        if r == 'eos':
            break
    print ' '.join(out[1:-1])

In [42]:
print_step = 1000
for i in range(4*len(inp)):
    ix = np.random.randint(0, len(inp))
    a = inp[ix]
    b = out[ix]
    feed = {incoming: a, decoder_target: b[1:], decoder_input : b[:-1]}
    for j in enc_layers[1:]:
        feed[j['initial']] = np.zeros(j['dim'])
        
    l, _, o = sess.run([loss, optimizer_op, outputs], feed_dict=feed)
    avg_loss += l
    if i % print_step == 0 and i != 0:
        print avg_loss / print_step
        avg_loss = 0
        saver.save(sess, 'checkpoint.ckpt')
        generate_out('what are you doing?')
        
    ix += 1

1.07981265824
great and and am am am am t t t t t t t t be be answer
0.699330968247
i what and am am am am am am am am t t t t t t t
0.633183161114
great hi and
0.542043530454
i i i i i i i i i i i i i i my just just my
0.470434768101
i hi i am i i i i i i i i i i am i my my
0.403817483911
i people and is am am is is is is is is is is is is is is
0.333009239973
i i i am i i i i i i don i m m be sore big code
0.271323352587
i hi am am am am was is don don don i he smell smell smell smell smell
0.219083797822
i hi am am i don don i i i don don flying m m m m m


In [49]:
# saver = tf.train.Saver()
saver.save(sess, 'model.ckpt')

'model.ckpt'

In [63]:
generate_out('do you know her')

the sky i i i i i i i i i i i i i i i i


In [48]:
data.head()

,0,1
0,Where do you live?,I live in Pasadena.
1,Where is Pasadena?,It's in California.
2,Is it in northern California?,No. It's in southern California.
3,Is Pasadena a big city?,It's pretty big.
4,"How big is ""pretty big""?","It has about 140,000 people."


In [131]:
[i_w[x] for x in np.argmax(inp[0], axis=1)]

['eos', 'where', 'do', 'you', 'live?', 'eos']